**중고차 가격 예측 데이터**
===

Attention
===

데이터 설명 : 중고차 가격 예측 데이터 (종속변수 :G3)   
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv   
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv   
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv   
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv   
데이터 출처 :https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv (참고, 데이터 수정)

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [4]:
x_train.nunique()

carID           4960
brand              9
model             90
year              24
transmission       4
mileage         3900
fuelType           5
tax               41
mpg              125
engineSize        37
dtype: int64

In [5]:
x_test.nunique()

carID           2672
brand              9
model             89
year              23
transmission       4
mileage         2226
fuelType           5
tax               34
mpg              107
engineSize        32
dtype: int64